In [2]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm

/Users/ded/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
import pickle
with open('data/gch-100-10-0.25-5.00.pkl', 'rb') as f:
  GCH, TH = pickle.load(f)
data = np.array(GCH)
I = len(GCH)
K = 10 # S sample periods
print("Captured:", I, "/ Sample Periods:",K)

Captured: 7 / Sample Periods: 10


In [11]:
tf.reset_default_graph()

#Input
f = tf.placeholder(tf.float64, shape=[I], name="first_time_seen")  
l = tf.placeholder(tf.float64, shape=[I], name="last_time_seen")  
u = tf.placeholder(tf.float64, shape=[I], name="count") 

# Parameters
U = tf.get_variable(dtype=tf.float64, shape=(), name = "unseen", initializer=tf.constant_initializer(100.)) 

ah = tf.get_variable("propensity_alpha", shape=(), dtype=tf.float64, initializer=tf.constant_initializer(np.sqrt(0.25))  )
a = tf.square(ah)

bh = tf.get_variable("propensity_beta",  shape=(), dtype=tf.float64, initializer=tf.constant_initializer(np.sqrt(5)) )
b = tf.square(bh)

#arrivals
lh0 = tf.constant(value=[0.4], name="arrivals_latent", shape=[1], dtype=tf.float64)
l0 = tf.sigmoid(lh0, name='arrivals_at_zero')
tmp_l = ((1-l0)/(K-1))
logit_lm = tf.log( tmp_l / (1 - tmp_l) )
lh = tf.concat( [lh0, logit_lm * tf.ones(K-1, dtype=tf.float64)], axis=0 )
lmd = tf.sigmoid(lh, name='arrivals')

#departures
mh0 = tf.get_variable("departures_latent", shape=[1], dtype=tf.float64, initializer=tf.constant_initializer(-0.4))
mh  = tf.ones( shape=[K], dtype=tf.float64 ) / (1.0*mh0)
mu  = tf.sigmoid(mh, name='departures')

## Masks
# Create masks using the values from f so that for each worker we add the lamda_s until the first appearance
s_mask = tf.sequence_mask(f+1, K)
s_mask = tf.cast(s_mask, tf.float64)

# Create masks using the values from l so that for each worker we add the mu_q from the first appearance until the end
q_mask = ~tf.sequence_mask(l, K)
q_mask = tf.cast(q_mask, tf.float64)

# This is a mask that only allows feasible combinations of s and q for a given worker i
# It is effectively the outer product of the s_mask and q_mask, for each worker
i_mask = tf.einsum("is,iq->isq", s_mask, q_mask)

v = np.zeros([K,K,K])
for s in np.arange(K):
    for q in np.arange(s,K):
        c1 = np.zeros(s)
        c2 = np.ones(q-s)
        c3 = np.zeros(K-q)
        v[s,q]= np.concatenate((c1, c2, c3))
v_mask = tf.constant(v)
# Calculating m_v(s,q) = Prod_s^q-1 mu_v
# mu_v[s,q]: Probability of surviving from period s to period q-1
# if q-1<s then probability is 1
mu_v = tf.pow(1-mu, v_mask)
mu_v = tf.reduce_prod(mu_v, axis=2)

# n = q - s + 1
d = np.zeros([K,K])
for s in range(K):
    for q in range(K):
        d[s,q] = q-s+1
        
# n-u
count = data[:, 2]
n_u = np.zeros([I,K,K])
for i in range(len(count)):
    n_u[i] = d - count[i]

n = tf.constant(np.clip(d,a_min=0,a_max=K+1))
n_u = tf.constant(np.clip(n_u,a_min=0,a_max=max(count)))

#Utilities
def transform_1d_to_3d(x, K, L):
    '''
    Takes vector x of dimensionality I and broadcast it, to return a 3d tensor,
    I x K x L, where the values of the 2d KxL matrix have the values of x[i]
    '''
    return tf.einsum("x,kl->xkl", x, tf.ones(shape=(K,L), dtype=tf.float64) )
def transform_2d_to_3d(xy, Z):
    '''
    Takes matrix  x of dimensionality X * Y  and broadcast it Z times, 
    to return a 3d tensor, Z x X x Y, where the values of the 2d KxL matrix have the values of x[i]
    '''
    return tf.einsum("xy,z->zxy", xy, tf.ones(shape=[Z], dtype=tf.float64) )

def get_risq():
    # tf.lgamma(n+1) 
    R_isq = transform_2d_to_3d( tf.lgamma( n+1 ), I)
    # - tf.lgamma(u+1) 
    R_isq -= transform_1d_to_3d( tf.lgamma(u+1), K, K )
    # - tf.lgamma(n-u+1)
    R_isq -= tf.lgamma( n_u + 1 )   
    # + tf.lgamma(u+a) 
    R_isq +=  transform_1d_to_3d(tf.lgamma(u+a), K, K)
    # + tf.lgamma(n-u+b)
    R_isq +=  tf.lgamma( n_u + b )
    # - tf.lgamma(n+a+b)
    R_isq -= transform_2d_to_3d( tf.lgamma( n +a+b ), I)
    # + tf.lgamma(a+b - tf.lgamma(a) -tf.lgamma(b)
    R_isq += tf.lgamma( a + b ) - tf.lgamma(a) - tf.lgamma(b )
    # The above is the computation of the log, so we take the exponent
    return tf.exp(R_isq)

R_isq = get_risq()
# Likelihoods of capture Li
LD = tf.einsum('s,sq,q->sq', lmd, mu_v, mu)
LD_isq = tf.einsum('sq,isq->isq', LD, i_mask)
Li = tf.einsum('isq,isq->i', LD_isq, R_isq)

# Likelihoods of no-capture L0
v0_mask = 1-v_mask[0]
LD0 = tf.multiply( LD  , v0_mask )

R0_sq = tf.exp(
  tf.lgamma(n + b) - tf.lgamma(n +a + b)
+ tf.lgamma(a + b) - tf.lgamma(b)
)

L0 = tf.reduce_sum(LD0 * R0_sq)

# Objective function
IT = tf.constant(I, dtype=tf.float64)
obj1 = tf.lgamma(IT+U+1) - tf.lgamma(U+1) - tf.lgamma(IT + 1)
obj2 = tf.reduce_sum(tf.log(Li))
obj3 = U * tf.log(L0)

objective = - (obj1 + obj2 +obj3) 

train_step = tf.train.AdamOptimizer(learning_rate=0.1).minimize(objective)

In [12]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
loss = []

feed_data = {
    f: data[:, 0],  
    l: data[:, 1], 
    u: data[:, 2]
}

t = tqdm(range(5000))

for epoch in t: 
    points = sess.run([train_step, objective, a, b, lmd, mu, U,i_mask, R_isq], feed_dict=feed_data)
    
    loss.append([epoch] + points)
    template = "{obj:10.3f} / {alpha:6.3f} / {beta:6.3f} / {arr:6.3f} / {dep:6.3f} / {u:6.3f}"
    
    if epoch % 10 == 0:
        status = template.format(
                        obj = points[1], 
                        alpha = points[2], 
                        beta = points[3],
                        arr = points[4][0],
                        dep = points[5][K-1],
                        u = points[6])
        t.set_description(status)

/Users/ded/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
    21.701 /  1.049 /  2.442 /  0.599 /  0.267 /  4.660: 100%|██████████| 5000/5000 [00:15<00:00, 318.10it/s]


In [10]:
points[4], points[5]   #departure, arrival

(array([0.59868766, 0.04459026, 0.04459026, 0.04459026, 0.04459026,
        0.04459026, 0.04459026, 0.04459026, 0.04459026, 0.04459026]),
 array([0.2665464, 0.2665464, 0.2665464, 0.2665464, 0.2665464, 0.2665464,
        0.2665464, 0.2665464, 0.2665464, 0.2665464]))

In [ ]:
df = pd.DataFrame(loss, columns = ['Train', 'Iteration', 'Loss', 'a', 'b', 'lambda', 'mu', 'U', 'i_mask', 'R_isq'])

In [ ]:
df.set_index('Iteration').Loss.plot()

In [ ]:
df.set_index('Iteration')[ ['a','b'] ].plot()

In [ ]:
df.set_index('Iteration')[ ['U'] ].plot()

In [ ]:
df['U'].iloc[-1] + I

In [ ]:
CH

In [ ]:
import tensorflow as tf

In [ ]:
tf.sequence_mask([1,2, 3], 5).eval()